In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI, OpenAI
from langchain_community.chains.graph_qa import cypher
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from neo4j import GraphDatabase

import pandas as pd
from neo4j import GraphDatabase
from datetime import datetime
import ast

# Initialize the Neo4j driver
uri = "neo4j+s://d1c92482.databases.neo4j.io"
username = "neo4j"
password = "H3mFJDVvmjPz1s5aDFi46XN8JFLGuYitPy6weie8stk"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [2]:
graph = Neo4jGraph(url=uri, username="neo4j", password=password)

# General

In [86]:
BASIC_CONTEXT = """
Some extra information about the graph database:
All Position Nodes:
(:Position {{POSITION_CODE: "MC", MAIN_POSITION: "Midfielder", ALIAS: "Number 8", POSITION_NAME: "Central Midfield"}})
(:Position {{POSITION_CODE: "DM", MAIN_POSITION: "Midfielder", ALIAS: "Number 6", POSITION_NAME: "Defensive Midfield"}})
(:Position {{POSITION_CODE: "ST", MAIN_POSITION: "Forward", ALIAS: "Number 9", POSITION_NAME: "Centre Forward"}})
(:Position {{POSITION_CODE: "DL", MAIN_POSITION: "Defender", ALIAS: NaN, POSITION_NAME: "Left Back"}})
(:Position {{POSITION_CODE: "LW", MAIN_POSITION: "Forward", ALIAS: NaN, POSITION_NAME: "Left Wing"}})
(:Position {{POSITION_CODE: "GK", MAIN_POSITION: "Goalkeeper", ALIAS: NaN, POSITION_NAME: "Goalkeeper"}})
(:Position {{POSITION_CODE: "MR", MAIN_POSITION: "Midfielder", ALIAS: NaN, POSITION_NAME: "Right Midfield"}})
(:Position {{POSITION_CODE: "DC", MAIN_POSITION: "Defender", ALIAS: NaN, POSITION_NAME: "Centre Back"}})
(:Position {{POSITION_CODE: "ML", MAIN_POSITION: "Midfielder", ALIAS: NaN, POSITION_NAME: "Left Midfield"}})
(:Position {{POSITION_CODE: "RW", MAIN_POSITION: "Forward", ALIAS: NaN, POSITION_NAME: "Right Wing"}})
(:Position {{POSITION_CODE: "DR", MAIN_POSITION: "Defender", ALIAS: NaN, POSITION_NAME: "Right Back"}})
(:Position {{POSITION_CODE: "AM", MAIN_POSITION: "Midfielder", ALIAS: "Number 10", POSITION_NAME: "Attacking Midfield"}})

All Strength nodes:
(:Strength {{STRENGTH: "Aerial duels"}})
(:Strength {{STRENGTH: "Ball control"}})
(:Strength {{STRENGTH: "Positioning"}})
(:Strength {{STRENGTH: "Tackling"}})
(:Strength {{STRENGTH: "Runs out"}})
(:Strength {{STRENGTH: "Discipline"}})
(:Strength {{STRENGTH: "High pressing"}})
(:Strength {{STRENGTH: "Finishing"}})
(:Strength {{STRENGTH: "High claims"}})
(:Strength {{STRENGTH: "Penalty taking"}})
(:Strength {{STRENGTH: "Passing"}})
(:Strength {{STRENGTH: "Long balls"}})
(:Strength {{STRENGTH: "Ball interception"}})
(:Strength {{STRENGTH: "Anchor play"}})
(:Strength {{STRENGTH: "Direct free kicks"}})
(:Strength {{STRENGTH: "Long shots saving"}})
(:Strength {{STRENGTH: "Reflexes"}})
(:Strength {{STRENGTH: "Long shots"}})
(:Strength {{STRENGTH: "Handling"}})
(:Strength {{STRENGTH: "Playmaking"}})
(:Strength {{STRENGTH: "Penalty saving"}})
(:Strength {{STRENGTH: "Error proneness"}})
(:Strength {{STRENGTH: "Consistency"}})
(:Strength {{STRENGTH: "Ground duels"}})

Some Tournament nodes:
(:Tournament {{TOURNAMENT_SLUG: "european-championship", COUNTRY: Europe, TOURNAMENT_ID: 1, TOURNAMENT_NAME: EURO}})
(:Tournament {{TOURNAMENT_SLUG: "trophee-des-champions", COUNTRY: France, TOURNAMENT_ID: 339, TOURNAMENT_NAME: Trophée des Champions}})
(:Tournament {{TOURNAMENT_SLUG: "european-championship-qualification", COUNTRY: Europe, TOURNAMENT_ID: 27, TOURNAMENT_NAME: EURO, Qualification}})
(:Tournament {{TOURNAMENT_SLUG: "ligue-1", COUNTRY: France, TOURNAMENT_ID: 34, TOURNAMENT_NAME: Ligue 1}})
(:Tournament {{TOURNAMENT_SLUG: "uefa-champions-league", COUNTRY: Europe, TOURNAMENT_ID: 7, TOURNAMENT_NAME: UEFA Champions League}})
(:Tournament {{TOURNAMENT_SLUG: "world-championship", COUNTRY: World, TOURNAMENT_ID: 16, TOURNAMENT_NAME: World Championship}})
(:Tournament {{TOURNAMENT_SLUG: "uefa-nations-league", COUNTRY: Europe, TOURNAMENT_ID: 10783, TOURNAMENT_NAME: UEFA Nations League}})
(:Tournament {{TOURNAMENT_SLUG: "world-championship-qual-uefa", COUNTRY: Europe, TOURNAMENT_ID: 11, TOURNAMENT_NAME: World Championship Qual. UEFA}})
(:Tournament {{TOURNAMENT_SLUG: "coupe-de-france", COUNTRY: France, TOURNAMENT_ID: 335, TOURNAMENT_NAME: Coupe de France}})
(:Tournament {{TOURNAMENT_SLUG: "coupe-de-la-ligue", COUNTRY: France, TOURNAMENT_ID: 333, TOURNAMENT_NAME: Coupe de la Ligue}})
(:Tournament {{TOURNAMENT_SLUG: "u19-european-championship", COUNTRY: Europe, TOURNAMENT_ID: 258, TOURNAMENT_NAME: U19 European Championship}})
(:Tournament {{TOURNAMENT_SLUG: "uefa-europa-leagu"e, COUNTRY: Europe, TOURNAMENT_ID: 679, TOURNAMENT_NAME: UEFA Europa League}})
(:Tournament {{TOURNAMENT_SLUG: "copa-america", COUNTRY: South America, TOURNAMENT_ID: 133, TOURNAMENT_NAME: Copa América}})
(:Tournament {{TOURNAMENT_SLUG: "supercopa-de-espana", COUNTRY: Spain, TOURNAMENT_ID: 213, TOURNAMENT_NAME: Supercopa de Espana}})
(:Tournament {{TOURNAMENT_SLUG: "world-championship-qual-conmebol", COUNTRY: South America, TOURNAMENT_ID: 295, TOURNAMENT_NAME: World Championship Qual. CONMEBOL}})
(:Tournament {{TOURNAMENT_SLUG: "laliga", COUNTRY: Spain, TOURNAMENT_ID: 8, TOURNAMENT_NAME: LaLiga}})
(:Tournament {{TOURNAMENT_SLUG: "club-world-championship", COUNTRY: World, TOURNAMENT_ID: 357, TOURNAMENT_NAME: Club World Championship}})
(:Tournament {{TOURNAMENT_SLUG: "uefa-super-cup", COUNTRY: Europe, TOURNAMENT_ID: 465, TOURNAMENT_NAME: UEFA Super Cup}})
(:Tournament {{TOURNAMENT_SLUG: "copa-del-rey", COUNTRY: Spain, TOURNAMENT_ID: 329, TOURNAMENT_NAME: Copa del Rey}})
(:Tournament {{TOURNAMENT_SLUG: "brasileirao-serie-a", COUNTRY: Brazil, TOURNAMENT_ID: 325, TOURNAMENT_NAME: Brasileirio Serie A}})
(:Tournament {{TOURNAMENT_SLUG: "bundesliga", COUNTRY: Germany, TOURNAMENT_ID: 35, TOURNAMENT_NAME: Bundesliga}})
(:Tournament {{TOURNAMENT_SLUG: "championship", COUNTRY: England, TOURNAMENT_ID: 18, TOURNAMENT_NAME: Championship}})
(:Tournament {{TOURNAMENT_SLUG: "trendyol-super-lig", COUNTRY: Turkey, TOURNAMENT_ID: 52, TOURNAMENT_NAME: Trendyol Süper Lig}})
(:Tournament {{TOURNAMENT_SLUG: "uefa-europa-conference-league", COUNTRY: Europe, TOURNAMENT_ID: 17015, TOURNAMENT_NAME: UEFA Europa Conference League}})
(:Tournament {{TOURNAMENT_SLUG: "serie-a", COUNTRY: Italy, TOURNAMENT_ID: 23, TOURNAMENT_NAME: Serie A}})
(:Tournament {{TOURNAMENT_SLUG: "premier-league", COUNTRY: England, TOURNAMENT_ID: 17, TOURNAMENT_NAME: Premier League}})
(:Tournament {{TOURNAMENT_SLUG: "olympic-games", COUNTRY: World, TOURNAMENT_ID: 436, TOURNAMENT_NAME: Olympic Games}})
(:Tournament {{TOURNAMENT_SLUG: "laliga-2", COUNTRY: Spain, TOURNAMENT_ID: 54, TOURNAMENT_NAME: LaLiga 2}})
(:Tournament {{TOURNAMENT_SLUG: "liga-portugal-betclic", COUNTRY: Portugal, TOURNAMENT_ID: 238, TOURNAMENT_NAME: Liga Portugal Betclic}})
(:Tournament {{TOURNAMENT_SLUG: "ligue-2", COUNTRY: France, TOURNAMENT_ID: 182, TOURNAMENT_NAME: Ligue 2}})
(:Tournament {{TOURNAMENT_SLUG: "premier-league", COUNTRY: Ukraine, TOURNAMENT_ID: 218, TOURNAMENT_NAME: Ukrainian Premier League}})
(:Tournament {{TOURNAMENT_SLUG: "liga-portugal-2", COUNTRY: Portugal, TOURNAMENT_ID: 239, TOURNAMENT_NAME: Liga Portugal 2}})
(:Tournament {{TOURNAMENT_SLUG: "eredivisie", COUNTRY: Netherlands, TOURNAMENT_ID: 37, TOURNAMENT_NAME: Eredivisie}})
(:Tournament {{TOURNAMENT_SLUG: "eerste-divisie", COUNTRY: Netherlands, TOURNAMENT_ID: 131, TOURNAMENT_NAME: Eerste Divisie}})
(:Tournament {{TOURNAMENT_SLUG: "primera-division", COUNTRY: Uruguay, TOURNAMENT_ID: 278, TOURNAMENT_NAME: Uruguayan Primera Division}})
(:Tournament {{TOURNAMENT_SLUG: "1-nl", COUNTRY: Croatia, TOURNAMENT_ID: 724, TOURNAMENT_NAME: 1. NL}})
(:Tournament {{TOURNAMENT_SLUG: "mls", COUNTRY: USA, TOURNAMENT_ID: 242, TOURNAMENT_NAME: MLS}})
(:Tournament {{TOURNAMENT_SLUG: "superliga", COUNTRY: Denmark, TOURNAMENT_ID: 39, TOURNAMENT_NAME: Danish Superliga}})
(:Tournament {{TOURNAMENT_SLUG: "liga-profesional-de-futbol", COUNTRY: Argentina, TOURNAMENT_ID: 155, TOURNAMENT_NAME: Liga Profesional de Futbol}})
(:Tournament {{TOURNAMENT_SLUG: "pro-league", COUNTRY: Belgium, TOURNAMENT_ID: 38, TOURNAMENT_NAME: Pro League}})
(:Tournament {{TOURNAMENT_SLUG: "bundesliga", COUNTRY: Austria, TOURNAMENT_ID: 45, TOURNAMENT_NAME: Bundesliga}})
(:Tournament {{TOURNAMENT_SLUG: "premier-liga", COUNTRY: Russia, TOURNAMENT_ID: 203, TOURNAMENT_NAME: Russian Premier League}})
(:Tournament {{TOURNAMENT_SLUG: "africa-cup-of-nations", COUNTRY: Africa, TOURNAMENT_ID: 270, TOURNAMENT_NAME: Africa Cup of Nations}})
(:Tournament {{TOURNAMENT_SLUG: "africa-cup-of-nations-qual", COUNTRY: Africa, TOURNAMENT_ID: 1848, TOURNAMENT_NAME: Africa Cup of Nations Qual.}})
(:Tournament {{TOURNAMENT_SLUG: "superliga", COUNTRY: Romania, TOURNAMENT_ID: 152, TOURNAMENT_NAME: Romanian Super Liga}})
(:Tournament {{TOURNAMENT_SLUG: "mozzart-bet-superliga", COUNTRY: Serbia, TOURNAMENT_ID: 210, TOURNAMENT_NAME: Mozzart Bet Superliga}})
(:Tournament {{TOURNAMENT_SLUG: "serie-b", COUNTRY: Italy, TOURNAMENT_ID: 53, TOURNAMENT_NAME: Serie B}})
(:Tournament {{TOURNAMENT_SLUG: "premiership", COUNTRY: Scotland, TOURNAMENT_ID: 36, TOURNAMENT_NAME: Scottish Premiership}})
(:Tournament {{TOURNAMENT_SLUG: "j1-league", COUNTRY: Japan, TOURNAMENT_ID: 196, TOURNAMENT_NAME: J1 League}})
(:Tournament {{TOURNAMENT_SLUG: "saudi-pro-league", COUNTRY: Saudi Arabia, TOURNAMENT_ID: 955, TOURNAMENT_NAME: Saudi Pro League}})
(:Tournament {{TOURNAMENT_SLUG: "eliteserien", COUNTRY: Norway, TOURNAMENT_ID: 20, TOURNAMENT_NAME: Eliteserien}})
(:Tournament {{TOURNAMENT_SLUG: "trendyol-1lig", COUNTRY: Turkey, TOURNAMENT_ID: 98, TOURNAMENT_NAME: Trendyol 1.Lig}})
NOTE: Ligue 1 of France, Bundesliga of Germany, La Liga of Spain, Premier League of England, and Serie A of Italy are referred to as 'Top 5 Leagues'
"""

In [87]:
from langchain_core.prompts.prompt import PromptTemplate

GENERAL_CYPHER_TEMPLATE = """Task:Generate Cypher statement to query a graph database. The database is of football (soccer) players data.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Only apply filters on objective attributes mentioned below. If the user question includes subjective descriptions also mentioned below, ignore them.
Subjective descriptions: fast, has pace, strong, etc.
Schema:
{schema}

"""

GENERAL_CYPHER_TEMPLATE_W_CONTEXT = GENERAL_CYPHER_TEMPLATE + BASIC_CONTEXT

GENERAL_CYPHER_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=GENERAL_CYPHER_TEMPLATE
)

GENERAL_CYPHER_PROMPT_W_CONTEXT = PromptTemplate(
    input_variables=["schema", "question"], template=GENERAL_CYPHER_TEMPLATE_W_CONTEXT
)

# ANALYZE

## ANALYSIS PROMPTS

In [26]:
ANALYSIS_CYPHER_COMMANDS = """
 The question will be to analyze and report on one football player that the user asks for. 
 The question will include the name of the player and you will search for it in PLAYER_NAME property of Player nodes. 
 
 The user may or may not ask for specific aspects of the player. 
 
 Below is a list of player aspects that user can ask for and relevant relationship types that holds data for that aspect.

 injuries (relationship) : HAD_INJURY {{INJURY_SEASON: STRING, INJURY_START_DATE: DATE, INJURY_FINISH_DATE: DATE, INJURY_DURATION: FLOAT, GAMES_MISSED_DURING_INJURY: FLOAT, CLUBS_DURING_INJURY: LIST}}
 ratings (relationship) : HAS_RATINGS {{RATING_COUNT: INTEGER, RATING_MEAN: FLOAT, RATING_STD_VAR: FLOAT}}
 attributes / characteristics (node properties) : Player {{ATTACKING: FLOAT, TECHNICAL: FLOAT, TACTICAL: INTEGER, DEFENDING: FLOAT, CREATIVITY: FLOAT}}
 Physical stats (relationship properties): PLAYED_IN_WITH_STATS {{totalDuelsWonPercentage , aerialDuelsWon , totalDuelsWon , duelLost , groundDuelsWonPercentage , groundDuelsWon}}
 Passing stats (relationship properties): PLAYED_IN_WITH_STATS {{expectedAssists , totalChippedPasses , totalCross , totalPasses , accuratePasses , totalLongBalls , accurateCrossesPercentage , accuratePassesPercentage , accurateOppositionHalfPasses , totalOwnHalfPasses , inaccuratePasses , accurateLongBallsPercentage , accurateChippedPasses , passToAssist , accurateCrosses , accurateLongBalls , accurateFinalThirdPasses , crossesNotClaimed}}
 Scoring stats (relationship properties): PLAYED_IN_WITH_STATS {{bigChancesMissed , attemptPenaltyTarget , attemptPenaltyPost , penaltyGoals , headedGoals , goals , penaltyConversion , goalsFromInsideTheBox , penaltiesTaken , goalsFromOutsideTheBox , goalsAssistsSum , scoringFrequency , goalConversionPercentage , goalKicks , hitWoodwork , goalsPrevented , freeKickGoal , setPieceConversion}}
 Shooting stats (relationship properties): PLAYED_IN_WITH_STATS {{shotsOffTarget , totalShots , shotsFromOutsideTheBox , blockedShots , shotsOnTarget , shotsFromInsideTheBox , shotFromSetPiece}}
 Defending stats (relationship properties): PLAYED_IN_WITH_STATS {{clearances , tacklesWon , tacklesWonPercentage , interceptions , errorLeadToShot , errorLeadToGoal , ownGoals , ballRecovery , tackles }}
 Goalkeeper stats (relationship properties): PLAYED_IN_WITH_STATS {{highClaims , savedShotsFromInsideTheBox , savesParried , penaltyFaced , saves , penaltySave , savesCaught , cleanSheet , successfulRunsOut , savedShotsFromOutsideTheBox , punches , goalsConceded , goalsConcededOutsideTheBox , goalsConcededInsideTheBox , goalsPrevented}}
 Possession / ball control stats (relationship properties): PLAYED_IN_WITH_STATS {{dispossessed , possessionLost , possessionWonAttThird}}
 Aggressiveness stats (relationship properties): PLAYED_IN_WITH_STATS {{yellowRedCards , redCards , fouls , yellowCards , directRedCards}}
 Aerial stats (relationship properties): PLAYED_IN_WITH_STATS {{aerialLost , aerialDuelsWonPercentage}}
 General stats (relationship properties): PLAYED_IN_WITH_STATS {{touches , totalRating , appearances , type , countRating , totwAppearances , rating , minutesPlayed , substitutionsOut , substitutionsIn , wasFouled}}
 Dribbling stats (relationship properties): PLAYED_IN_WITH_STATS {{dribbledPast , successfulDribbles , successfulDribblesPercentage}}
 Attacking  (relationship properties): PLAYED_IN_WITH_STATS {{offsides , penaltyWon}}

 If question asks for specific aspects generate the query so that only the node and / or relationship properties are turned by the query. 
 For example,

 Question: 
 How is Kylian Mbappe's physicality in 2023-2024 season
 Query:
 MATCH (p:Player {{PLAYER_NAME: 'Kylian Mbappe'}})-[r:PLAYED_IN_WITH_STATS]->(t:Tournament)
 RETURN
  p, 
  r.totalDuelsWonPercentage AS totalDuelsWonPercentage,
  r.aerialDuelsWon AS aerialDuelsWon,
  r.totalDuelsWon AS totalDuelsWon,
  r.duelLost AS duelLost,
  r.groundDuelsWonPercentage AS groundDuelsWonPercentage,
  r.groundDuelsWon AS groundDuelsWon,
  t;

 The question will include the name of the player and you will search for it in PLAYER_NAME property of Player nodes.
 

 The question is:
 {question}

 IMPORTANT NOTE: If the question does not include a football player, then just write 'False' and nothing else. Obey this rule at all cost.
 """

ANALYSIS_CYPHER_TEMPLATE = GENERAL_CYPHER_TEMPLATE + ANALYSIS_CYPHER_COMMANDS
ANALYSIS_CYPHER_TEMPLATE_W_CONTEXT = GENERAL_CYPHER_TEMPLATE_W_CONTEXT + ANALYSIS_CYPHER_COMMANDS

ANALYSIS_CYPHER_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=ANALYSIS_CYPHER_TEMPLATE
)

ANALYSIS_CYPHER_PROMPT_W_CONTEXT = PromptTemplate(
    input_variables=["schema", "question"], template=ANALYSIS_CYPHER_TEMPLATE_W_CONTEXT
)

In [27]:
ANALYSIS_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers for football player analysis.
The information part contains the provided information that you must use to construct an answer.
The information will include various data. Make use of it, analyze the data and report you findings in your answer.
Also the information may include multiple instances of the same data field. Account for all instances. Sum, aggregate them if necessary.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
You can use internal knowledge to support your answer ONLY if the question explicity asks you to. If question does not say that, do ONLY and ONLY use context information.

If the provided information is empty, say that you don't know the answer.
Information:
{context}

Question: {question}
Helpful Answer:"""

ANALYSIS_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=ANALYSIS_QA_TEMPLATE
)

## ANALYSIS CHAIN

In [ ]:
question = "Evaluate Javi Galan's defending, passing, physical, and aerial stats in La Liga and champions league in the 23/24 season"

In [ ]:
analysis_cypher_chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    prompt=ANALYSIS_CYPHER_PROMPT_W_CONTEXT
)

In [ ]:
analysis_cypher_response = analysis_cypher_chain.invoke(
    {
        "schema": graph.schema,
        "question": question
    }
)

In [ ]:
print(analysis_cypher_response['text'])

In [ ]:
analysis_context = graph.query(analysis_cypher_response['text'][10:-3])

In [ ]:
analysis_qa_chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model='gpt-4o'),
    prompt=ANALYSIS_QA_PROMPT
)

In [ ]:
analysis_qa_response = analysis_qa_chain.invoke(
    {
        "context": analysis_context,
        "question": question
    }
)

# COMPARE

## COMPARE PROMPTS

In [71]:
COMPARISON_CYPHER_COMMANDS = """
 The question will be to analyze and report on multiple (two or more) football player that the user asks for. 
 The question will include the names of the players and you will search for them in PLAYER_NAME property of Player nodes. 
 
 The user may or may not ask for specific aspects of the players. 
 
 Below is a list of player aspects that user can ask for and relevant relationship types that holds data for that aspect.

 injuries (relationship) : HAD_INJURY {{INJURY_SEASON: STRING, INJURY_START_DATE: DATE, INJURY_FINISH_DATE: DATE, INJURY_DURATION: FLOAT, GAMES_MISSED_DURING_INJURY: FLOAT, CLUBS_DURING_INJURY: LIST}}
 ratings (relationship) : HAS_RATINGS {{RATING_COUNT: INTEGER, RATING_MEAN: FLOAT, RATING_STD_VAR: FLOAT}}
 attributes / characteristics (node properties) : Player {{ATTACKING: FLOAT, TECHNICAL: FLOAT, TACTICAL: INTEGER, DEFENDING: FLOAT, CREATIVITY: FLOAT}}
 Physical stats (relationship properties): PLAYED_IN_WITH_STATS {{totalDuelsWonPercentage , aerialDuelsWon , totalDuelsWon , duelLost , groundDuelsWonPercentage , groundDuelsWon}}
 Passing stats (relationship properties): PLAYED_IN_WITH_STATS {{expectedAssists , totalChippedPasses , totalCross , totalPasses , accuratePasses , totalLongBalls , accurateCrossesPercentage , accuratePassesPercentage , accurateOppositionHalfPasses , totalOwnHalfPasses , inaccuratePasses , accurateLongBallsPercentage , accurateChippedPasses , passToAssist , accurateCrosses , accurateLongBalls , accurateFinalThirdPasses , crossesNotClaimed}}
 Scoring stats (relationship properties): PLAYED_IN_WITH_STATS {{bigChancesMissed , attemptPenaltyTarget , attemptPenaltyPost , penaltyGoals , headedGoals , goals , penaltyConversion , goalsFromInsideTheBox , penaltiesTaken , goalsFromOutsideTheBox , goalsAssistsSum , scoringFrequency , goalConversionPercentage , goalKicks , hitWoodwork , goalsPrevented , freeKickGoal , setPieceConversion}}
 Shooting stats (relationship properties): PLAYED_IN_WITH_STATS {{shotsOffTarget , totalShots , shotsFromOutsideTheBox , blockedShots , shotsOnTarget , shotsFromInsideTheBox , shotFromSetPiece}}
 Defending stats (relationship properties): PLAYED_IN_WITH_STATS {{clearances , tacklesWon , tacklesWonPercentage , interceptions , errorLeadToShot , errorLeadToGoal , ownGoals , ballRecovery , tackles }}
 Goalkeeper stats (relationship properties): PLAYED_IN_WITH_STATS {{highClaims , savedShotsFromInsideTheBox , savesParried , penaltyFaced , saves , penaltySave , savesCaught , cleanSheet , successfulRunsOut , savedShotsFromOutsideTheBox , punches , goalsConceded , goalsConcededOutsideTheBox , goalsConcededInsideTheBox , goalsPrevented}}
 Possession / ball control stats (relationship properties): PLAYED_IN_WITH_STATS {{dispossessed , possessionLost , possessionWonAttThird}}
 Aggressiveness stats (relationship properties): PLAYED_IN_WITH_STATS {{yellowRedCards , redCards , fouls , yellowCards , directRedCards}}
 Aerial stats (relationship properties): PLAYED_IN_WITH_STATS {{aerialLost , aerialDuelsWonPercentage}}
 General stats (relationship properties): PLAYED_IN_WITH_STATS {{touches , totalRating , appearances , type , countRating , totwAppearances , rating , minutesPlayed , substitutionsOut , substitutionsIn , wasFouled}}
 Dribbling stats (relationship properties): PLAYED_IN_WITH_STATS {{dribbledPast , successfulDribbles , successfulDribblesPercentage}}
 Attacking  (relationship properties): PLAYED_IN_WITH_STATS {{offsides , penaltyWon}}

 If user question asks for specific aspects generate the query so that only the node and / or relationship properties of those aspects are turned by the query.
 IMPORTANT NOTE: Some properties are in multiple aspects categories. If the user selects those aspects or does not select an aspect at all, write those properties ONLY ONCE to prevent duplication 


 For every player name in the question write a seperate query and return them in a list

 For example,
 Question: 
 Compare Erling Haaland's performance in Premier League and Kylian Mbappe's in Ligue 1
 Query:
 [
    "MATCH (p:Player)-[r:PLAYED_IN_WITH_STATS]->(t:Tournament) WHERE p.PLAYER_NAME = 'Kylian Mbappe' AND t.TOURNAMENT_NAME = 'Ligue 1' RETURN p, r.totalDuelsWonPercentage AS totalDuelsWonPercentage, r.aerialDuelsWon AS aerialDuelsWon, r.totalDuelsWon AS totalDuelsWon, r.duelLost AS duelLost, r.groundDuelsWonPercentage AS groundDuelsWonPercentage, r.groundDuelsWon AS groundDuelsWon, r.expectedAssists AS expectedAssists, r.totalChippedPasses AS totalChippedPasses, r.totalCross AS totalCross, r.totalPasses AS totalPasses, r.accuratePasses AS accuratePasses, r.totalLongBalls AS totalLongBalls, r.accurateCrossesPercentage AS accurateCrossesPercentage, r.accuratePassesPercentage AS accuratePassesPercentage, r.accurateOppositionHalfPasses AS accurateOppositionHalfPasses, r.totalOwnHalfPasses AS totalOwnHalfPasses, r.inaccuratePasses AS inaccuratePasses, r.accurateLongBallsPercentage AS accurateLongBallsPercentage, r.accurateChippedPasses AS accurateChippedPasses, r.passToAssist AS passToAssist, r.accurateCrosses AS accurateCrosses, r.accurateLongBalls AS accurateLongBalls, r.accurateFinalThirdPasses AS accurateFinalThirdPasses, r.crossesNotClaimed AS crossesNotClaimed, r.bigChancesMissed AS bigChancesMissed, r.attemptPenaltyTarget AS attemptPenaltyTarget, r.attemptPenaltyPost AS attemptPenaltyPost, r.penaltyGoals AS penaltyGoals, r.goals AS goals, r.penaltyConversion AS penaltyConversion, r.goalsFromInsideTheBox AS goalsFromInsideTheBox, r.penaltiesTaken AS penaltiesTaken, r.goalsFromOutsideTheBox AS goalsFromOutsideTheBox, r.goalsAssistsSum AS goalsAssistsSum, r.scoringFrequency AS scoringFrequency, r.goalConversionPercentage AS goalConversionPercentage, r.goalKicks AS goalKicks, r.hitWoodwork AS hitWoodwork, r.goalsPrevented AS goalsPrevented, r.freeKickGoal AS freeKickGoal, r.setPieceConversion AS setPieceConversion, r.shotsOffTarget AS shotsOffTarget, r.totalShots AS totalShots, r.shotsFromOutsideTheBox AS shotsFromOutsideTheBox, r.blockedShots AS blockedShots, r.shotsOnTarget AS shotsOnTarget, r.shotsFromInsideTheBox AS shotsFromInsideTheBox, r.shotFromSetPiece AS shotFromSetPiece, r.clearances AS clearances, r.tacklesWon AS tacklesWon, r.tacklesWonPercentage AS tacklesWonPercentage, r.interceptions AS interceptions, r.errorLeadToShot AS errorLeadToShot, r.errorLeadToGoal AS errorLeadToGoal, r.ownGoals AS ownGoals, r.ballRecovery AS ballRecovery, r.tackles AS tackles, r.highClaims AS highClaims, r.savedShotsFromInsideTheBox AS savedShotsFromInsideTheBox, r.savesParried AS savesParried, r.penaltyFaced AS penaltyFaced, r.saves AS saves, r.penaltySave AS penaltySave, r.savesCaught AS savesCaught, r.cleanSheet AS cleanSheet, r.successfulRunsOut AS successfulRunsOut, r.savedShotsFromOutsideTheBox AS savedShotsFromOutsideTheBox, r.punches AS punches, r.goalsConceded AS goalsConceded, r.goalsConcededOutsideTheBox AS goalsConcededOutsideTheBox, r.goalsConcededInsideTheBox AS goalsConcededInsideTheBox, r.dispossessed AS dispossessed, r.possessionLost AS possessionLost, r.possessionWonAttThird AS possessionWonAttThird, r.yellowRedCards AS yellowRedCards, r.redCards AS redCards, r.fouls AS fouls, r.yellowCards AS yellowCards, r.directRedCards AS directRedCards, r.aerialLost AS aerialLost, r.aerialDuelsWonPercentage AS aerialDuelsWonPercentage, r.touches AS touches, r.totalRating AS totalRating, r.appearances AS appearances, r.type AS type, r.countRating AS countRating, r.totwAppearances AS totwAppearances, r.rating AS rating, r.minutesPlayed AS minutesPlayed, r.substitutionsOut AS substitutionsOut, r.substitutionsIn AS substitutionsIn, r.wasFouled AS wasFouled, r.dribbledPast AS dribbledPast, r.successfulDribbles AS successfulDribbles, r.successfulDribblesPercentage AS successfulDribblesPercentage, r.offsides AS offsides, r.penaltyWon AS penaltyWon, t;",
    "MATCH (p:Player)-[r:PLAYED_IN_WITH_STATS]->(t:Tournament) WHERE p.PLAYER_NAME = 'Erling Haaland' AND t.TOURNAMENT_NAME = 'Premier League' RETURN p, r.totalDuelsWonPercentage AS totalDuelsWonPercentage, r.aerialDuelsWon AS aerialDuelsWon, r.totalDuelsWon AS totalDuelsWon, r.duelLost AS duelLost, r.groundDuelsWonPercentage AS groundDuelsWonPercentage, r.groundDuelsWon AS groundDuelsWon, r.expectedAssists AS expectedAssists, r.totalChippedPasses AS totalChippedPasses, r.totalCross AS totalCross, r.totalPasses AS totalPasses, r.accuratePasses AS accuratePasses, r.totalLongBalls AS totalLongBalls, r.accurateCrossesPercentage AS accurateCrossesPercentage, r.accuratePassesPercentage AS accuratePassesPercentage, r.accurateOppositionHalfPasses AS accurateOppositionHalfPasses, r.totalOwnHalfPasses AS totalOwnHalfPasses, r.inaccuratePasses AS inaccuratePasses, r.accurateLongBallsPercentage AS accurateLongBallsPercentage, r.accurateChippedPasses AS accurateChippedPasses, r.passToAssist AS passToAssist, r.accurateCrosses AS accurateCrosses, r.accurateLongBalls AS accurateLongBalls, r.accurateFinalThirdPasses AS accurateFinalThirdPasses, r.crossesNotClaimed AS crossesNotClaimed, r.bigChancesMissed AS bigChancesMissed, r.attemptPenaltyTarget AS attemptPenaltyTarget, r.attemptPenaltyPost AS attemptPenaltyPost, r.penaltyGoals AS penaltyGoals, r.goals AS goals, r.penaltyConversion AS penaltyConversion, r.goalsFromInsideTheBox AS goalsFromInsideTheBox, r.penaltiesTaken AS penaltiesTaken, r.goalsFromOutsideTheBox AS goalsFromOutsideTheBox, r.goalsAssistsSum AS goalsAssistsSum, r.scoringFrequency AS scoringFrequency, r.goalConversionPercentage AS goalConversionPercentage, r.goalKicks AS goalKicks, r.hitWoodwork AS hitWoodwork, r.goalsPrevented AS goalsPrevented, r.freeKickGoal AS freeKickGoal, r.setPieceConversion AS setPieceConversion, r.shotsOffTarget AS shotsOffTarget, r.totalShots AS totalShots, r.shotsFromOutsideTheBox AS shotsFromOutsideTheBox, r.blockedShots AS blockedShots, r.shotsOnTarget AS shotsOnTarget, r.shotsFromInsideTheBox AS shotsFromInsideTheBox, r.shotFromSetPiece AS shotFromSetPiece, r.clearances AS clearances, r.tacklesWon AS tacklesWon, r.tacklesWonPercentage AS tacklesWonPercentage, r.interceptions AS interceptions, r.errorLeadToShot AS errorLeadToShot, r.errorLeadToGoal AS errorLeadToGoal, r.ownGoals AS ownGoals, r.ballRecovery AS ballRecovery, r.tackles AS tackles, r.highClaims AS highClaims, r.savedShotsFromInsideTheBox AS savedShotsFromInsideTheBox, r.savesParried AS savesParried, r.penaltyFaced AS penaltyFaced, r.saves AS saves, r.penaltySave AS penaltySave, r.savesCaught AS savesCaught, r.cleanSheet AS cleanSheet, r.successfulRunsOut AS successfulRunsOut, r.savedShotsFromOutsideTheBox AS savedShotsFromOutsideTheBox, r.punches AS punches, r.goalsConceded AS goalsConceded, r.goalsConcededOutsideTheBox AS goalsConcededOutsideTheBox, r.goalsConcededInsideTheBox AS goalsConcededInsideTheBox, r.dispossessed AS dispossessed, r.possessionLost AS possessionLost, r.possessionWonAttThird AS possessionWonAttThird, r.yellowRedCards AS yellowRedCards, r.redCards AS redCards, r.fouls AS fouls, r.yellowCards AS yellowCards, r.directRedCards AS directRedCards, r.aerialLost AS aerialLost, r.aerialDuelsWonPercentage AS aerialDuelsWonPercentage, r.touches AS touches, r.totalRating AS totalRating, r.appearances AS appearances, r.type AS type, r.countRating AS countRating, r.totwAppearances AS totwAppearances, r.rating AS rating, r.minutesPlayed AS minutesPlayed, r.substitutionsOut AS substitutionsOut, r.substitutionsIn AS substitutionsIn, r.wasFouled AS wasFouled, r.dribbledPast AS dribbledPast, r.successfulDribbles AS successfulDribbles, r.successfulDribblesPercentage AS successfulDribblesPercentage, r.offsides AS offsides, r.penaltyWon AS penaltyWon, t;"
 ]

 The question will include the name of the player and you will search for it in PLAYER_NAME property of Player nodes.

 The question is:
 {question}

 IMPORTANT NOTE: If the question does not include multiple football player names, then just write 'False' and nothing else. Obey this rule at all cost.

"""

COMPARISON_CYPHER_TEMPLATE = GENERAL_CYPHER_TEMPLATE + COMPARISON_CYPHER_COMMANDS
COMPARISON_CYPHER_TEMPLATE_W_CONTEXT = GENERAL_CYPHER_TEMPLATE_W_CONTEXT + COMPARISON_CYPHER_COMMANDS

COMPARISON_CYPHER_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=COMPARISON_CYPHER_TEMPLATE
)

COMPARISON_CYPHER_PROMPT_W_CONTEXT = PromptTemplate(
    input_variables=["schema", "question"], template=COMPARISON_CYPHER_TEMPLATE_W_CONTEXT
)

In [72]:
COMPARISON_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers for football player analysis.
The information part contains the provided information that you must use to construct an answer.
The information will include various data. Make use of it, analyze the data and report you findings in your answer.
The information will consist of stats of multiple football players and you will be asked to compare them in some way. You can group the stats as passing, attacking, defense etc. to better format your answer.
Also the information may include multiple instances of the same data field. Account for all instances. Sum, aggregate them if necessary.
The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
You can use internal knowledge to support your answer ONLY if the question explicity asks you to. If question does not say that, do ONLY and ONLY use context information.

If the provided information is empty, say that you don't know the answer.
Information:
{context}

Question: {question}
Helpful Answer:"""

COMPARISON_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=COMPARISON_QA_TEMPLATE
)

## COMPARE CHAIN

In [73]:
question = "Compare Arthur and Weston McKennie's Serie A 23/24 performances" # in in terms of passing, defending, and physicality

In [74]:
comparison_cypher_chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    prompt=COMPARISON_CYPHER_PROMPT_W_CONTEXT
)

In [75]:
comparison_cypher_response = comparison_cypher_chain.invoke(
    {
        "schema": graph.schema,
        "question": question
    }
)

In [76]:
print(comparison_cypher_response['text'])

[
    "MATCH (p:Player)-[r:PLAYED_IN_WITH_STATS]->(t:Tournament) WHERE p.PLAYER_NAME = 'Arthur' AND t.TOURNAMENT_NAME = 'Serie A' AND r.SEASON = '23/24' RETURN p, r.totalDuelsWonPercentage AS totalDuelsWonPercentage, r.aerialDuelsWon AS aerialDuelsWon, r.totalDuelsWon AS totalDuelsWon, r.duelLost AS duelLost, r.groundDuelsWonPercentage AS groundDuelsWonPercentage, r.groundDuelsWon AS groundDuelsWon, r.expectedAssists AS expectedAssists, r.totalChippedPasses AS totalChippedPasses, r.totalCross AS totalCross, r.totalPasses AS totalPasses, r.accuratePasses AS accuratePasses, r.totalLongBalls AS totalLongBalls, r.accurateCrossesPercentage AS accurateCrossesPercentage, r.accuratePassesPercentage AS accuratePassesPercentage, r.accurateOppositionHalfPasses AS accurateOppositionHalfPasses, r.totalOwnHalfPasses AS totalOwnHalfPasses, r.inaccuratePasses AS inaccuratePasses, r.accurateLongBallsPercentage AS accurateLongBallsPercentage, r.accurateChippedPasses AS accurateChippedPasses, r.passToAss

In [77]:
comparison_cypher_list = ast.literal_eval(comparison_cypher_response['text'])

In [78]:
print(comparison_cypher_list[0])

MATCH (p:Player)-[r:PLAYED_IN_WITH_STATS]->(t:Tournament) WHERE p.PLAYER_NAME = 'Arthur' AND t.TOURNAMENT_NAME = 'Serie A' AND r.SEASON = '23/24' RETURN p, r.totalDuelsWonPercentage AS totalDuelsWonPercentage, r.aerialDuelsWon AS aerialDuelsWon, r.totalDuelsWon AS totalDuelsWon, r.duelLost AS duelLost, r.groundDuelsWonPercentage AS groundDuelsWonPercentage, r.groundDuelsWon AS groundDuelsWon, r.expectedAssists AS expectedAssists, r.totalChippedPasses AS totalChippedPasses, r.totalCross AS totalCross, r.totalPasses AS totalPasses, r.accuratePasses AS accuratePasses, r.totalLongBalls AS totalLongBalls, r.accurateCrossesPercentage AS accurateCrossesPercentage, r.accuratePassesPercentage AS accuratePassesPercentage, r.accurateOppositionHalfPasses AS accurateOppositionHalfPasses, r.totalOwnHalfPasses AS totalOwnHalfPasses, r.inaccuratePasses AS inaccuratePasses, r.accurateLongBallsPercentage AS accurateLongBallsPercentage, r.accurateChippedPasses AS accurateChippedPasses, r.passToAssist AS 

In [79]:
comparison_context = [graph.query(cypher_query) for cypher_query in comparison_cypher_list]

In [83]:
comparison_qa_chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model='gpt-4o'),
    prompt=COMPARISON_QA_PROMPT
)

In [84]:
comparison_qa_response = comparison_qa_chain.invoke(
    {
        "context": comparison_context,
        "question": question
    }
)

In [85]:
comparison_qa_response['text']

"Let's compare the performances of Arthur and Weston McKennie in the Serie A 23/24 season across various aspects:\n\n### Passing\n- **Arthur**:\n  - Total Passes: 1466\n  - Accurate Passes: 1350 (92.09% accuracy)\n  - Accurate Opposition Half Passes: 774\n  - Accurate Final Third Passes: 342\n  - Total Long Balls: 132 (68.94% accuracy)\n  - Total Crosses: 21 (42.86% accuracy)\n  - Total Chipped Passes: 62 (39 accurate)\n\n- **Weston McKennie**:\n  - Total Passes: 941\n  - Accurate Passes: 760 (80.77% accuracy)\n  - Accurate Opposition Half Passes: 490\n  - Accurate Final Third Passes: 267\n  - Total Long Balls: 45 (62.22% accuracy)\n  - Total Crosses: 59 (23.73% accuracy)\n  - Total Chipped Passes: 65 (38 accurate)\n\n### Attacking\n- **Arthur**:\n  - Goals: 2\n  - Assists: 3\n  - Total Shots: 12 (6 on target)\n  - Shots from Inside the Box: 2\n  - Shots from Outside the Box: 10\n  - Big Chances Missed: 0\n\n- **Weston McKennie**:\n  - Goals: 0\n  - Assists: 7\n  - Total Shots: 23 (7 o

# FIND

## FIND PROMPTS

In [95]:
FIND_CYPHER_COMMANDS = """
If the following descriptions or their synonyms / paraphrases are mentioned in the question, look for the provided properties and/or relationships. The variables in user question will be denoted between <> For example, if user question is "plays for Real Madrid", template is "plays for <name of a football team>":
"aged <age number>" : PLAYER_BIRTH_DATE of Player node. The current year is 2024. Subtrack and find the age. If age is below 25, consider the player young.
"Played in <name of a tournament>" : PLAYED_IN_WITH_STATS relationship exists to the Tournament node specified. Property to match is TOURNAMENT_SLUG. 
"Plays for <name of a football team>" : PLAYS_FOR relationship exists to the Team node specified. Property to match is TEAM_SLUG
"worth <a number of value>" : PLAYER_MARKET_VALUE property of Player node. If the user question number is written out like 1m, 1M, 1 million, put the actual number in the query, which is 1000000. If user says "around X", it means "between X-2000000 and X+2000000"
"tall" : PLAYER_HEIGHT property of Player node is above 180
Note: Normalise user input for names. For example, if user input is "plays for Çaykur Rizespor", query should have "t:Team {{TEAM_NAME : "caykur-rizespor"}}"

After applying these filters, return all properties and relationships of the matching nodes.
IMPORTANT NOTE: If the user query includes subjective descriptions such as fast, strong, intelligent, etc. ignore them

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
After generating the query according to user question criteria, return all relationships of those nodes.
Examples: Here are a few examples of generated Cypher statements for particular questions:

Question: Players with good physique that play for Real Madrid as a defender?
Query:
MATCH (p:Player)-[:PLAYS_FOR]->(t:Team {{TEAM_NAME: 'Real Madrid'}})
MATCH (p)-[:PLAYS_IN_THE_POSITION_OF]->(pos:Position {{MAIN_POSITION: 'Defender'}})
MATCH (p)-[r]->(n)
RETURN p

Question: Find intelligent number 8 players worth around 10 million from top 5 leagues aged between 20-27
Query:
MATCH (p)-[:PLAYS_IN_THE_POSITION_OF]->(pos:Position {{POSITION_NAME: 'Central Midfield'}})
WHERE 8000000 < p.PLAYER_MARKET_VALUE < 12000000  
AND p.LEAGUE_NAME IN ["LaLiga", "SerieA", "PremierLeague", "Bundesliga", "Ligue1"] 
AND date("1997-01-01") < p.PLAYER_BIRTH_DATE < date("2005-01-01")
RETURN p

The question is:
{question}
"""

FIND_CYPHER_TEMPLATE = GENERAL_CYPHER_TEMPLATE + FIND_CYPHER_COMMANDS
FIND_CYPHER_TEMPLATE_W_CONTEXT = GENERAL_CYPHER_TEMPLATE_W_CONTEXT + FIND_CYPHER_COMMANDS

FIND_CYPHER_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=FIND_CYPHER_TEMPLATE
)

FIND_CYPHER_PROMPT_W_CONTEXT = PromptTemplate(
    input_variables=["schema", "question"], template=FIND_CYPHER_TEMPLATE_W_CONTEXT
)

In [98]:
FIND_QA_TEMPLATE = """You are an assistant that helps to form nice and human understandable answers for football player analysis.
The information part contains the provided information that you must use to construct an answer.
The information will include various data. Make use of it, analyze the data and report you findings in your answer.
The information will consist of some players and their attributes. List them with brief descriptions of the players. You can group the players based on the criteria mentioned in the question.
For example, if question includes "from top 5 league", you can group the players based on the league they are playing.

The provided information is authoritative, you must never doubt it or try to use your internal knowledge to correct it.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
You can use internal knowledge to support your answer ONLY if the question explicity asks you to. If question does not say that, do ONLY and ONLY use context information.

If the provided information is empty, say that you don't know the answer.
Information:
{context}

Question: {question}
Helpful Answer:"""

FIND_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=FIND_QA_TEMPLATE
)

## FIND CHAIN

In [99]:
find_cypher_chain = LLMChain(
    llm=ChatOpenAI(temperature=0, model="gpt-4o"),
    prompt=FIND_CYPHER_PROMPT_W_CONTEXT
)

In [100]:
question = "Find left back players from top 5 league worth around 10 million "

In [101]:
find_cypher_response = find_cypher_chain.invoke(
    {
        "schema": graph.schema,
        "question": question
    }
)

In [102]:
print(find_cypher_response['text'])


```cypher
MATCH (p:Player)-[:PLAYS_IN_THE_POSITION_OF]->(pos:Position {POSITION_NAME: 'Left Back'})
WHERE 8000000 < p.PLAYER_MARKET_VALUE < 12000000  
AND p.LEAGUE_NAME IN ["LaLiga", "Serie A", "Premier League", "Bundesliga", "Ligue 1"]
MATCH (p)-[r]->(n)
RETURN p, r, n
```


# Bellingham vs Musiala